# Assignment 2

### Ferenc Huszár, Nic Lane and Neil Lawrence

### 23rd February 2021

Welcome to the second assignment for the Deep Neural Networks module. In this assignment you will explore some of the model architectures we talked about in the second half of lectures.

There are 230 marks given in total for this second assessment ($\approx 70\%$ of the total of 330 marks for this course), broken into three groups:
* (A) 75 marks for the guided questions on Residual Networks
* (B) 75 marks for the guided questions on RNNs
* (C) 80 marks for a mini-project of your choice

You can choose whichever mini-project you want to attempt, we give you a range of options. If you would like to do more of these, that's fine, but will only mark one, whichever appears fist in your submitted notebook. These tasks are a bit more exploratoy, you don't have to go overboard on them. Equally, if you struggle to get them to work the way you wanted/expected, please document what you tried and what you learned.

# A: Residual Networks

In these questions we will work with residual networks (ResNets). In order to save time, we won't be training these models ourselves. Pytorch provides pre-*trained* weights for a range of commonly used models in the `torchvision` package, including ResNets of various depth. Most of the pretrained ResNets available are slightly improved versions of those proposed in 'Deep Residual Learning for Image Recognition' by [He et al (2015)](https://arxiv.org/pdf/1512.03385.pdf).

Note: you can request colab to run a GPU by going to Runtime -> Change runtime type. However, since we are only going to work with single images, it is unlikely this will buy you significant speedup.


## Setting up: Blas the border collie

To start, we will look at whether ResNets of different length can correctly classify my friend's dog Blas as a Border Collie.

In [ ]:
!wget https://caballerojose.com/images/blas.jpg -P blas_the_dog/test

### Loading an image

We will use the Python Imaging Library (PIL) to load the image. Colab then displays the image in-line if it is the output of the last line in the cell. Hello Blas!

In [ ]:
from PIL import Image
img = Image.open('blas_the_dog/test/blas.jpg')
img.resize((800,535))

### Transformations

ConvNets expect input images to be of a certain size, otherwise the dimensions of the tensors within the network don't work out. We have to resize this image so it is of size $224\times 224$ as the network expects. Secondly, when training neural networks, we often normalise the inputs (substract the mean and divide by the standard deviation). As this model was trained on ImageNet, we have to use the mean and standard deviation from this dataset. Thankfully, we don't have to compute this, as it's included in the [pytorch documentation](https://pytorch.org/vision/0.8/models.html) (as well as elsewhere on the internet).

We will use `transforms` from the `torchvision` module to apply these transformations to the image. We also convert from `PIL.Image` to `torch.Tensor` in the process:


In [ ]:
from torchvision import transforms

transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225]
                                )])

img_t = transform(img)

We can display the transformed image by converting it back to `PIL.Image`. We can see that the image was downsized, cropped to it's centre square region, and the colours are messed up due to normalization.

In [ ]:
transforms.ToPILImage()(img_t)

### Loading  the model

Now we are going to load the pre-trained 18-layer ResNet model from the `torchvision` library.

In [ ]:
from torchvision import models

resnet18 = models.resnet18(pretrained=True)

Let's see how the model classifies Blas, the border collie. For this, we have to turn the image tensor into a minibatch of 1 example, which practically means adding an extra dimension to the tensor. This can be done using the `unsqueeze` function.

In [ ]:
x = img_t.unsqueeze(0)
y = resnet18(x)
y.size()

The model's output is a vector of 1000 logits, one for each class in the ImageNet dataset. To interpret this, we have to know what the ImageNet classes are. Also, the logits themselves are less human-readable, so we are going to turn them into probabilities using a `softmax` function. This helper function below will do this for us.

In [ ]:
import requests
import json
from torch.nn.functional import softmax

imagenet_labels_url = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
response = requests.get(imagenet_labels_url)
imagenet_class_names = json.loads(response.text)

def top_5_classes(y, class_names = None):
  if class_names==None:
    imagenet_labels_url = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
    response = requests.get(imagenet_labels_url)
    class_names = json.loads(response.text)
  p = softmax(y[0,:], dim=0)
  values, indices = p.topk(5)
  return [(class_names[index], value) for index, value in zip(indices.detach().numpy(), values.detach().numpy())]

In [ ]:
top_5_classes(y)

Oops, the model doesn't seem to recognise Blas. It thinks Blas is a bucket, plunger or hook. This is because in order to use the pretrained models in pytorch, we have to set them to evaluation mode. When training deep neural networks, sometimes the networks work differently at training and test time. For example, when using a technique called dropout, random units in the network may be dropped at training time, but at test time, all units are used and their output is scaled. Similarly, when using batch normalisation, the network works differently in training time and at test time. To set models to evaluation mode we can use the `eval` function.

In [ ]:
resnet18.eval()
y = resnet18(x)
top_5_classes(y)

OK, getting there. At least the model now recognises Blas as a dog, but misses the specific breed. The correct class would be a Border Collie.

## Question A.1: Trying different pretrained networks

Now try different models from `torchvision` and see which ones can classify Blas correctly. In particular, check out deeper versions of ResNets. Use the [documentation](https://pytorch.org/vision/0.8/models.html) to find out what other networks are available. Use the code blocks below, and the text field to summarise what you found. (For full marks, try multiple ResNets, and at least two different non-ResNets).

*10 marks*

### Answer A.1

Please use this cell to summarise which networks you tried.

In [ ]:
# please add code for question A.1 here. You may add further code cells below.

## Inspecting and modifying networks

For the next question, you will need to modify the pre-trained networks, and you will need to look inside them. Below, we share a few things that will help you learn how models work. First, let's look at what the `resnet18` architecture looks like and how that architecture is represented in pytorch. You can print the model to see a text description of it:

In [ ]:
print(resnet18)

Here, modules are listed from top to bottom in order of how they are executed.In a pytorch ResNet, residual blocks are organised into `layers`, each of which contain sequences of residual blocks. You will notice that:
* the number of channels keeps doubling at each layer, in this case from 128 -> 256 -> 512.
* at the same time, the figure size is halved with each layer. The image size is halved by `MaxPool2d` layers as well as strided convolutions where the stride is `2`.
* a `resnet18` is built out of `BasicBlock` building blocks, which are the simple residual block proposed by [He et al, (2015)](https://arxiv.org/abs/1512.03385) which I showed in the lecture. Deeper ResNets, such as `resnet50` use a more complicated `Bottleneck` component instead.
* in pytorch, the linear and nonlinear parts of a layer are separate, i.e. the convolution and subsequent ReLU activation are handled as separate layers.

To access the second convolution layer from the second residual block at the third layer, you can write the following:

In [ ]:
resnet18.layer3[1].conv2

And you can access parameters of layers like so.

In [ ]:
resnet18.conv1.weight.size()

We can visualise the convolution kernels on the first layer using the code below. In addition to showing how to access weights, you may find some of this code useful as an example of using the `matplotlib.pylab` package for plotting.

In [ ]:
import numpy as np
from matplotlib import pylab as plt

f = plt.figure(figsize = (8,8))

weights = resnet18.conv1.weight.detach().numpy()
#normalising weights so a value of 0 becomes gray, and colours stay within [0,1]
weights = weights/(np.abs(weights).max())/2+0.5

#there are 64 convolution kernels, which we will arrange along an 8x8 grid
for i, w in enumerate(weights):
  plt.subplot(8, 8, i+1)
  # we move the colour channel axis to the end, this is where imshow expects it
  plt.imshow(np.moveaxis(w, 0, -1));
  plt.axis('off')

In pytorch, you can inspect what each layer of the network does by using forward hooks. A forward hook is a python function that gets called whenever the layer is evaluated (i.e. data is passed through the layer). The forward hook function receives both the input and the output of the module. In pytorch, you can add a forward hook using the register_forward_hook function. 

For example, in the code below, I attach a hook to a layer, which simply prints something out. Instead of attaching the hook to the resnet18, I first create a `deepcopy` of the model, so the hook gets attached to the copy, but not to the `resnet18` model which I may use later.

In [ ]:
import copy

def useless_hook(module, input, output):
  print('The hook was called.')

model = copy.deepcopy(resnet18)
model.layer3[1].conv2.register_forward_hook(useless_hook)
y = model(x)
y = model(x)

## Question A.2: Inspecting layers in neural networks

This question has a subtasks:
1. print the number of parameters in each of the operations/layers of the `ResNet18`, and count how many there are in total.
1. print the *shape of the output tensor* of the second convolution operation (`conv2`) within every single residual block in a `ResNet18`. *Tip:* you can use a forward hook.
1. Extract the activation (output) of a layer of your choice within the ResNet18, and visualise the activations using `matplotlib.pylab.imshow`. *Tip:* you can use a forward for this here, too.
1. Visualise the convolution weights (also called kernels) in the first convolution layer of a pretrained AlexNet and a VGG19 network. Compare it to what we obtained the ResNet above. Document the similarities and differences do you notice.


*Tips:*
* you can use `copy.deepcopy` to create copies of models.
* You can use `tensor.size()` to get the sape.
* you have to use `tensor.detach().numpy()` to detach a pytorch tensor from the computational graph and convert it to a numpy array.
* for plotting, you can use the `matplotlib.pylab` module. The function `imshow` can be usedd to plot activation maps as images, `axis('off')` is handy for removing axis labels. `subplot` allows you to create grids of multiple images in one figure, finally `figure(figsize=(x, y))` lets you change the overall size of the figure.
* on a pytorch module you can call `named_children` and `name_parameters` to iterate over layers and parameters, respectively.

*15 marks*

### Answer A.2

please provide your answers in code cells below (you may use multiple code cells).

## Editing models

Models can be edited rather flexibly in pytorch. Underlying a pytorch `Module` are python `OrderedDict` objects. These are ordered lists of key-value pairs. Each submodule and each parameter has a name, which is the key. Members of the dictionay can be accessed as attributes of the `Module`. Adding a new parameter or layer to a module will add a new key-value pair to this dictionary.

Let's try adding a new operation to our resnet18, which converts the logits to probabilities. We will do this by replacing the final operation by a `Sequential` module, which chains multiple operations together.

In [ ]:
from torch.nn import Softmax, Sequential

model = copy.deepcopy(resnet18)
model.fc = Sequential(
    model.fc,
    Softmax(dim=1)
    )

In [ ]:
#let's see what the model looks like now
print(model)

Let's check the model still works and that it outputs a vector of probabilities which therefore sums to one.

In [ ]:
y = model(x)
print(y.sum())

We can also replace existing layers with other layers. Here, for example, I replace the first convolution layer with a randomly initialized one of the same dimensions. We can check what effect this has on the classification of Blas:

In [ ]:
from torch.nn import Conv2d

model = copy.deepcopy(resnet18)
model.conv1 = Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
y = model(x)
top_5_classes(y)

Or, if we replace the convolutional layer by the convolutions from the VGG19 network, followed by a MaxPooling, to account for the difference in stride, the network becomes pretty certain that Blas is, in fact, spider web, or possibly a type of spider.

In [ ]:
from torch.nn import MaxPool2d

vgg19 = models.vgg19(pretrained=True)

model = copy.deepcopy(resnet18)
model.conv1 = Sequential(
      vgg19.features[0],
      MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False))
y=model(x)
top_5_classes(y)

## Question A.3: Dropping layers in ResNet15

This question is composed of several tasks:
1. Delete the second `BasicBlock` from `layer4` of the ResNet18, and see how the network's predictions change. 
1. Delete the first `BasicBlock` from `layer4` of the ResNet18, and see how the network's predictions change. (Note, this is a bit morer involved than the previous one).
1. Delete the first `BasicBlock` from `layer2` of the ResNet18, and see how the network's predictions change.

Tips:
* There are several ways for removing a ResNet block:
  * The easiest perhaps is to replace the entire block with a `torch.nn.Identity`. This works for blocks where the output and input featuremap size are the same. However, some of the residual blocks also reduce the size of the image (strided convolutions, or Max Pooling), so replacing by the Identity won't work. This is why the second task is a bit harder than the first. To find out what you have to do in the second task, look closely at the difference between the two `BasicBlock` instances within `layer4`. To understand how this block works, you can also look at the [source code](https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py#L67).
  * Since the activations are all ReLU, you can also ignore the effect of layers by setting appropriate weights and biases to zero. This is because $\operatorname{ReLU}(0z).= 0$. However, if you are trying to use this method, be aware of batch normalization layers - you also want to remove the effect of those.
  * You will notice that the layers of ResNets are of `torch.nn.Sequential` type. You can drop an element from this by creating a new `Sequential` instead with one operation removed.
* You can solve this task using a few lines of code only.

*15 marks*

### Answers A.3

Please provide your answer in the code blocks below.

In [ ]:
#1. dropping layer4 second block

In [ ]:
#2. dropping layer4 first block

In [ ]:
#2. dropping layer2 first block

## A convenience function: flattening ResNets

Residual blocks in pytorch ResNets are organised into 4 layers. I found this rather cumbersome for the next exercise we are going to do. In order to make ResNets easier to work with I created the convenience function below which can *flatten* the layer hierarchy of pytorch resnet models, merging the 4 layers into one, called `residual_blocks`. After flattening, instead of accessing `model.layer1[0]` you can do `model.residual_blocks[0]`.

In [ ]:
from torch import nn
from collections import OrderedDict

def flatten_resnet(resnet_model):
  """
  Takes a pytorch resnet model and returns a copy of the same model in a 
  flattened format, where the residulal blocks are not divided into four layers
  but are instead all in a residual blocks component.
  """
  model = copy.deepcopy(resnet_model)
  residual_blocks = nn.Sequential(*model.layer1,
                *model.layer2,
                *model.layer3,
                *model.layer4)
  return nn.Sequential(OrderedDict([
     ('conv1', model.conv1),
     ('bn1', model.bn1),
     ('relu', model.relu),
     ('maxpool', model.maxpool),
     ('residual_blocks', residual_blocks),
     ('avgpool', model.avgpool),
     ('flatten', nn.Flatten()),
     ('fc', model.fc)
  ]))

## Question A.4: Dropping layers in ResNet50

Load a pretrained ResNet50, and evaluate the model's predictions on the photo of Blas. Does it get the breed correct? (Border Collie).

Look at the structure of the ResNet50. Identify the `Bottleneck` blocks which are used instead of `BasicBlock` building blocks in deeper ResNets. These are still arranged into 4 'layers', and there are 16 of them in total.

Drop each of the 16 `Bottleneck` blocks from the ResNet50 (one at a time, not all at once), and make predictions with each of the perturbed networks. Plot the predicted probability of the Border Collie class (ImageNet class number 232) as a function of the index of the layer dropped. Compare the probabilities you get with the probability for the unperturbed model. What do you see?

*20 marks*

### Answers A.4

Please add text in this box, and use as many code cells below as you'd like.

## Question A.5: Permuting and Repeating Layers

Now, let's see what happens if we permute or repeat layers in the ResNet.

1. Permute layers: Swap `resnet50.layer2[1]` and `resnet50.layer2[3]` in the network (so that instead of the second `Bottleneck` unit in `layer2` you use the fourth and vice versa). Make predictions with the modified network. How did perturbing the model this way modify the predictions?
1. Repeat a residual block in the ResNet. Apply the second `Bottleneck` block of `layer2` of the ResNet twice instead of just once, making the model deeper. Make predictions with the modified model. What do you find?
1. Now repeat the layer 2, 3, 4, 5 times. Do the predictions keep improving?

*15 marks*

### Answers A.5

Please add text in this box, and use as many code cells below as you'd like.

# B: Recurrent Networks

In this part of the assignment, we will train simple recurrent networks to detect messages hidden in binary strings.

## Backgroud

### Creating the dataset

Below, I included a function which creates a pytorch `Dataset` we will use in the questions below. 


In [ ]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

def get_binary_dataset(num_datapoints, signature_length=20, random_length=20):
  '''
  Creates a labelled dataset of `num_datapoints` binary sequences. Each sequence
  is of total length `signature_length + random_length`. Positive examples have
  a signature string of length `signature_length` embedded starting at a random
  location within the string. The signature is a binary string of alternating 1s
  and 0s. Negative examples are random bitstrings. They may contain the
  signature by chance, although the probability of this diminishes as the
  `signature_legth` increases.

  The returned dataset contains positive examples first, followed by negatives.
  When using minibatch-SGD, the dataset has to be shuffled before broken into
  minibatches.
 
  Attributes
  ----------
  num_datapoints : int
    Number of datapoints to be generated. Half of the datapoints will be
    positive, the rest negative. If `num_datapoints==1` then the dataset will
    contain a single positive example.
  signature_length : int
    Length of the signature string
  random_length : int
    Number of random bits in the sequence in addition to the signature sequence.
  
  Returns
  -------
  torch.utils.data.TensorDataset
    Pytorch dataset containing the strings and their binary labels.
  '''
  num_positives = num_datapoints - num_datapoints // 2
  y = np.array([[1.0]]*num_positives + [[0.0]]*(num_datapoints - num_positives))
  X = np.random.randint(2, size=(num_datapoints, signature_length + random_length))
  signature = np.array((signature_length//2)*[1.0, 0.0])
  for i in range(num_positives):
    j = np.random.randint(random_length)
    X[i, j:(j+signature_length)] = signature
  X = torch.tensor(X, dtype=torch.float)[:, :, None]
  y = torch.tensor(y)
  return TensorDataset(X, y)

Below, I visualise a dataset of $100$ positive and $100$ negative sequences of lenth $40$ each. In the visualisation below, positive examples appear on top, negatives in the bottom half. Black pixels show $1$s, white pixels show $0$s. You can see the signature sequence `10101010101010101010` embedded in the positive examples at random locations.

*Notes:* You can see how I used `spy` to visualise a sparse matrix. This function can be used to show non-zero values in a matrix. The pytorch `TensorsDataset` object has a `tensors` attribute which contains the input and output tensors.

In [ ]:
from matplotlib import pylab as plt

ds = get_binary_dataset(200,
                        signature_length=20,
                        random_length=20)

plt.figure(figsize=(4, 16))
plt.spy(ds.tensors[0][:, :, 0].numpy())
plt.axis('off');

## Building a classifier

Now we're going to build a simple RNN-based classifier for classifying binary sequences. We are going to feed the sequence into an RNN, then extract the hidden state activations at the end of the sequence, finally apply a linear transformation to calculate the classifier output.

An RNN module returns two outputs, as explained in [the documentation](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html). We have to select a layer which selects the second output - which is the activation of the hidden state at the end of the sequence - rearranges and flattens it so it's ready to be fed into a linear layer.

*Note:* By applying a further `sigmoid`, this classifier output can be turned into a probability, like it was the case with `softmax` the ResNet example. We are not going to make the sigmoid part of the network, as training will be more numerically stable this way

In [ ]:
class extract_last_cell(nn.Module):
  '''Pytorch module which extracts and flattens the last hidden state of an RNN.
  '''
  def forward(self,x):
    _ , out = x
    return out.permute(1,0,2).flatten(start_dim=1)

def get_basic_RNN_classifier():
  return nn.Sequential(
    nn.RNN(input_size=1, hidden_size=20, num_layers=2, batch_first=True),
    extract_last_cell(),
    nn.Linear(in_features = 20*2, out_features=1)
  )

## Question B.1: Evaluating the loss

Evaluate the log-loss of a randomly initialized classifier on minibatches of training examples from the dataset we defined before.

*Tips:*
* you can use [`pytorch/utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) to sample minibatches of training data from the dataset
* or, you can also access the input and output tensors of a `TensorDataset` directly via the `tensors` attribute
* use [`torch.nn.BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html) or [`torch.nn.functional.binary_cross_entropy_with_logits`](https://pytorch.org/docs/stable/nn.functional.html)

*10 marks*

### Answer B.1

pleasse add answer in the code cell(s) below:

## Training the classifier

In the previous assignment we wrote a training loop to show how optimizers work in pytorch. For this assignment we are going to use ignite, a flexible module for reusable training loops in pytorch. We will install ignite, and then I provide a function you can use to train models on the sequence data.

In [ ]:
!pip install pytorch-ignite

In [ ]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss

def fit_model_basic(model, training_dataset, test_dataset):
  '''Fits a pytorch model to the training dataset using stochastic gradient
  descent and tracks progress on a test set.

  Arguments
  ---------
  model : pytorch.nn.Module
    pytorch model that can be evaluated on input tensor in the dataset, and
    returns a tensor that matches the shape of the output tensor.
  training_dataset : pytorch.util.data.Dataset
    used to train the model
  test_dataset : pytorch.util.data.Dataset
    used to evaluate the performance of the model at the end of each epoch

  Retuns
  ------
  pytorch.nn.Module
    the model after training
  '''

  train_loader = DataLoader(
      training_dataset,
      batch_size=100,
      shuffle=True
  )
  val_loader = DataLoader(
      test_dataset,
      batch_size=1000,
      shuffle=False,
  )

  optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
  criterion = nn.BCEWithLogitsLoss()

  trainer = create_supervised_trainer(model, optimizer, criterion)

  def thresholded_output_transform(output):
      y_pred, y = output
      y_pred = torch.heaviside(y_pred, values=torch.zeros(1))
      return y_pred, y

  val_metrics = {
      "accuracy": Accuracy(thresholded_output_transform),
      "bce": Loss(criterion)
  }
  evaluator = create_supervised_evaluator(model, metrics=val_metrics)

  @trainer.on(Events.ITERATION_COMPLETED(every=10))
  def log_training_loss(trainer):
      print(f"Epoch[{trainer.state.epoch}] Loss: {trainer.state.output:.2f}")

  @trainer.on(Events.EPOCH_COMPLETED)
  def log_training_results(trainer):
      evaluator.run(train_loader)
      metrics = evaluator.state.metrics
      print(f"Training Results - Epoch: {trainer.state.epoch}  Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['bce']:.2f}")

  @trainer.on(Events.EPOCH_COMPLETED)
  def log_validation_results(trainer):
      evaluator.run(val_loader)
      metrics = evaluator.state.metrics
      print(f"Validation Results - Epoch: {trainer.state.epoch}  Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['bce']:.2f}")

  trainer.run(train_loader, max_epochs=5)

  return model

The code above looks rather long and complicated, but that is because we ask ignite to log the progress of training in a very verbose way. Ignite uses python decorators (like the `@trainer.on(Events.EPOCH_COMPLETED)`) bits. You will find plenty of tutorials online if you google 'python decorators' if you want to learn more about them. In ignite, these decorators are used to register functions which are called when various events happen during training.

For example, `@trainer.on(Events.ITERATION_COMPLETED(every=10))` tells ignite to execute the function that's below it every 10th iteration is completed. You will find that this training loop is largely a copy of the example code provided in the [ignite master documentation](https://pytorch.org/ignite/quickstart.html#code).

One difference is that we use the `thresholded_output_transform` to tarnsform the network's output before [`Accuracy`](https://pytorch.org/ignite/metrics.html#ignite.metrics.Accuracy) metric can be evaluated. Accuracy compares binary predictions to binary labels. But our network outputs `logits` instead of binary predictors. To turn these into binary decisions, we have to determine whether the logits are above or below $0$, which is what the `heaviside` function does.

Ignite can do much more than what we ask it to do here. It's a flexible way of creating reproducible training loops for neural network training. It can handle distributed training over several machines, etc.

Let's now use our training loop to train a classifier on the binary dataset we created:

In [ ]:
ds_train = get_binary_dataset(5000, signature_length=10, random_length=10)
ds_test = get_binary_dataset(1000, signature_length=10, random_length=10)
rnn_classifier = get_basic_RNN_classifier()

fit_model_basic(rnn_classifier, ds_train, ds_test)

Oh no! The error doesn't seem like the loss is reduced. Let's try to fix this.



## Question B.2: Improving the SGD code

1. Create a new version of `fit_model_basic` function, call it `fit_model`. You will use this function in subsequent questions. Your function should take additional arguments allowing you to easily specify key hyperparameters of optimization: learning rate, batchsize, whatever else you find important.
1. Tune hyperparameters and change things until training works reliably. Consider [changing the optimizer](https://pytorch.org/docs/stable/optim.html) from vanilla SGD: you may try SGD with momentum, `Adam`, etc. You should be able to make this example work, i.e. reach $90\%+$ average accuarcy within a few seconds of training.

*Tips*:
 * You can try different learning rates, such as the [Karpathy constant](https://twitter.com/karpathy/status/801621764144971776). This is a joke, learning rates are usually problem-specific. Values between $0.1$ and $0.001$ are often tried.
 * If you want, you can run grid search: running training for different values of hyperparameters, and checkind which one produces the best results. If doing so, consider spacing parameter values logarithmically rather than linearly. I.e. it makes more sense to try learning rates $[0.001, 0.01, 0.1]$ than to try $[0.001, 0.002, 0.003, \ldots]$.

*15 marks*

### Answer B.2

Please modify the code in the cells below. You may use this text box to summarise the changes you made.

In [ ]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss

def fit_model(model, training_dataset, test_dataset):
  '''Fits a pytorch model to the training dataset using stochastic gradient
  descent and tracks progress on a test set.

  Arguments
  ---------
  model : pytorch.nn.Module
    pytorch model that can be evaluated on input tensor in the dataset, and
    returns a tensor that matches the shape of the output tensor.
  training_dataset : pytorch.util.data.Dataset
    used to train the model
  test_dataset : pytorch.util.data.Dataset
    used to evaluate the performance of the model at the end of each epoch

  Retuns
  ------
  pytorch.nn.Module
    the model after training
  '''

  train_loader = DataLoader(
      training_dataset,
      batch_size=100,
      shuffle=True
  )
  val_loader = DataLoader(
      test_dataset,
      batch_size=1000,
      shuffle=False,
  )

  optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
  criterion = nn.BCEWithLogitsLoss()

  trainer = create_supervised_trainer(model, optimizer, criterion)

  def thresholded_output_transform(output):
      y_pred, y = output
      y_pred = torch.heaviside(y_pred, values=torch.zeros(1))
      return y_pred, y

  val_metrics = {
      "accuracy": Accuracy(thresholded_output_transform),
      "bce": Loss(criterion)
  }
  evaluator = create_supervised_evaluator(model, metrics=val_metrics)

  @trainer.on(Events.ITERATION_COMPLETED(every=10))
  def log_training_loss(trainer):
      print(f"Epoch[{trainer.state.epoch}] Loss: {trainer.state.output:.2f}")

  @trainer.on(Events.EPOCH_COMPLETED)
  def log_training_results(trainer):
      evaluator.run(train_loader)
      metrics = evaluator.state.metrics
      print(f"Training Results - Epoch: {trainer.state.epoch}  Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['bce']:.2f}")

  @trainer.on(Events.EPOCH_COMPLETED)
  def log_validation_results(trainer):
      evaluator.run(val_loader)
      metrics = evaluator.state.metrics
      print(f"Validation Results - Epoch: {trainer.state.epoch}  Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['bce']:.2f}")

  trainer.run(train_loader, max_epochs=5)

  return model

In [ ]:
classifier = fit_model(get_basic_RNN_classifier(), ds_train, ds_test)

## Question B.3: Test on longer sequences

Now that you successfully trained an RNN, let's test how well it works when we use it on longer sequences than what it was trained on. Does it have the ability to remember? Remember we trained on datasets with `signature_length=10, random_length=10`, that is a total length of `20` binary symbols.

* create 5 test datasets with `signature_length` of $10$ but with different `random_length` parameters (e.g. $10$, $20$, $30$, $50$ and $100$)
* evaluate accuracy of your trained model on each of these test regimes - (without retraining!)
* plot the performance you achieve for different test sequence lengths
* describe what you find

*Tips:*
* look inside the `fit_model` function for tips on evaluating models using ignite components.

*20 marks*

### Answers B.3

Please complete the task in code cells below.

## Question B.4: Better models

Remember what we learned about vanilla RNN's ability to remember information over long time range. Pick a different RNN cell type that should have better ability to remember, and define a new network that uses this cell type. 
* Modify the function below to define an improved RNN classifier
* train an instance of this better RNN model on the same data we trained the previous classifier on
* repeat the analysis from Question 3 using your new model, and contrast your new architecture's performance with the vanilla RNN from before.

*15 marks*

### Answers B.4

Please edit the code below and add as many code cells as you would like to complete this task.

In [ ]:
def get_better_RNN_classifier():
  '''A better RNN architecture for solving the signature detection problem.
  '''
  return nn.Sequential(
    nn.RNN(input_size=1, hidden_size=20, num_layers=2, batch_first=True),
    extract_last_cell(),
    nn.Linear(in_features = 20*2, out_features=1)
  )

In [ ]:
classifier = fit_model(get_better_RNN_classifier(), ds_train, ds_test)

## Question B.5: Visualise memorization

Now we are going to visualize which parts of the input the trained architecture pays attention to when making its decision. This is going to be similar in spirit to [this distill post](https://distill.pub/2019/memorization-in-rnns/).
* take your pre-trained RNN classifier from the previous example
* generate a random positive input sequence (one that has a signature of length $10$ in it) and form an input tensor to the model.
* calculate the gradient of the model's output with respect to the input.

*Tips:*
* remember how we used `requires_grad` and `backward` and the `grad` attribute in the lecture to calculate the gradients of an RNN's hidden state with respect to the input sequence.
* remember the `TensorDataset` has a `tensors` argument which you can use to extract the input and output tensors from the dataset.

*15 marks*

### Answers B.5

Please add your solution to the code blocks below.

## C Mini-Projects

Please **choose one** of the proposed mini-project questions below. Add as many text and code cells under the heading as you'd like. You may attempt more than one, but we will only mark one: please write below which mini-project you'd like to get marked on:

**I would like my solution for mini-project __________ to be marked.**

## C.1: Adversarial examples for ResNets

Adversarial examples for image classifiers are images which have been imperceptibly changed such that the classifier's output changes. For example, below we see an image of a panda, which the network correctly classifies as panda (left), but when we add a small amount of structured noise (middle), and feed the modified image (right) into the classifier, it now classifies it is a gibbon with high confidence.

![](https://openai.com/content/images/2017/02/adversarial_img_1.png)

In this mini-project, your task is to generate an adversarial examples for a pretrained convolutional neural network of your choice (VGG19, ResNet18 or ResNet50 are good choices). Use the image of Blas as a starting point. Change the image of Blas imperceptibly so that your network misclassifies him as a bus or sea slug, or any other arbitrary ImageNet class which is clearly not a dog. I recommend using the *fast gradient sign* method described on page 3 of [(Szegedy et al, 2013)](https://arxiv.org/pdf/1412.6572.pdf). *(40 marks)*

For full marks, explore the topic a bit more, example questions you can ask:
* Can you deliberately control what class you get after the adversarial attack? *(~20 marks)*
* Does the adversarial example generated for one network fool other networks, too? *(~20 marks for non-minimal investigation and good writeup)*
* Load another image. Does the adversarial noise generated for one image work to fool the network when added to a different image *(~20 marks for for non-minimal investigation and good writeup)*
* Can you generate adversarial examples without access to gradients, only using the forward pass of the network (this is called black-box adversarial attack)? *(~40 marks)*

*Tips:*
* The *fast gradient sign* method relies on gradients of the network's output with respect to pixel values in image space: you are looking for the smallest change to pixel values which maximally change the classifier's outputs. Remember how we did this using `requires_gradients` and `backward` in the RNN example.
* To illustrate the change is imperceptible, you want to apply the change to the untransformed image, i.e. before the normalisation is applied. The normalization is itself differentiable, so my advice is to make the normalization part of the network itself.

*80 marks*

## C.2: Extend on binary sequence learning from questions B

Building on the quided question set B on Recurrent Neural Networks, explore what RNNs and different models can do on the binary string datasets. You can try the following
* Make the task more difficult (for this, extend `get_binary_dataset`):
  * Can you successfully train networks on longer training sequences? *(~10 marks)*
  * What happens if you add noise to the data, e.g. randomly flip bits?  *(~20 marks)*
  * Embed more than one copy of the signature sequence, can you train a network to count them?  *(~20 marks)*
  * Can networks memorise messages emmbedded in random bit strings. For example, can you train a Seq2Seq network recover k random bits that immediately follow the signature sequence?  *(~40 marks)*
* Try different architectures:
  * What architectures, other than RNNs could solve this problem well? Do you need deep architectures? Design another model and train it on the binary dataset. Can you get them to solve the problem? Can you illustrate the pros and cons of different architectures for this problem? *(~30 marks for one architecture, up to ~70 marks for trying and comparing substantially different architectures in a good writeup)*
* Hand design neural networks that solve this problem:
  * This problem can be solved with simple algorithms. Implement these algorithms in neural hardware, i.e. by manually setting weights in a neural network. *(~40 marks)*

*Tips:*
  * For counting, you might want to think about what an appropriate loss function is. You can set this up as a multiclass classification problem and use [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html). Think about how the network's output has to change.
  * When changing architectures substantially, you may find that the same optimization hyperparameters no longer work.
  * pytorch RNN modules return two outputs, if you want to use attention, you want the first output (cell values at the top layer at all timesteps), for vanilla seq2seq or classification you need the second output (hidden states at all layers at the final timestep)

*80 marks*

##C.3: Model compression via Weight Matrix Decomposition

Model compression is the generic name used to refer to techniques applied to model architectures so that they can be more system resource efficient -- especially at inference time. (If you wish to read more about this area feel free to consult this [tutorial](https://arxiv.org/abs/1703.09039), but note it is not needed in this assignment.) In this mini-project, you will have the opportunity to explore one of these methods that involves decomposing a single large weight matrix of an architecture into a series of smaller matrices to reduce memory and compute needs, while still maintaining accuracy levels of the original model. 

A popular early approach of this type involved using matrix factorization methods (like SVD) and applying them to layers within an architecture where the weight matrix causes performance bottlenecks for the whole model. In this case a single weight matrix is broken down into two smaller matrices, the product of which approximates the original larger matrix.

This mini-project is comprised by the following following steps and questions below. We ask that you work with a pretrained [AlexNet](https://papers.nips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf) and the ImageNet dataset. We will be using SVD to perform the matrix factorization:

* Applying SVD decomposition to AlexNet's fully-connected (dense) layers *(~20 marks).*
  * Perform SVD on the weight matrix of the first fully-connected layer of AlexNet. The weight matrix of the layer will be decomposed into two matrices ($U$ and $V$ by SVD) *(note, feel free to use a library implementation of SVD)*.
  * Under which conditions will the use of the $U$ and $V$ matrices above, when used in place of the weight matrix, result in parameter savings? 
  * Implement a new layer type that you can use to replace the fully-connected layers within AlexNet. This new layer type should replace the single weight matrix in a conventional fully-connected layer with the $U$ and $V$ matrices. Include a parameter in your layer that determines how many singular vectors are used (parameter $k$ within SVD) *(note, you only need to implement the forward-pass of this layer given how we use it next)*.
  * Replace both fully-connected layers within AlexNet with your new layer type developed above. Experiment by varying the values of $k$ and see how this changes the test accuracy of the model *(note, to speed-up experiments only compute accuracy for a 128-example subset of ImageNet test data, also only use $k$ values that result in an overal reduction of parameters in the model)*.
* Implementing the SVD decomposition as two seperate layers *(~40 marks).*
  * Show that the new layer you implemented above, to support the decomposition of weights, can also be viewed (and implemented) as two seperate layers. Provide an illustration to show this is possible, and also provide a new implementation of a layer type to support this *(note, again at this stage only the forward pass of the implementation is required)*.
  * Describe the implications to the per-layer working set memory at inference time (defined during lectures) of implementing weight decomposition as two layers compare to when it is implemented as a single layer? 
* Re-training your SVD decompositions *(~20 marks).*
  * Implement the backwards pass of the two-layer approach to supporting SVD-based weight decomposition.
  * Re-train the AlexNet model with both fully-connected layers using this two-layer implementation *(note, you must re-train all model parameters, not just those related to the fully-connected layers)*. Did test accuracy improve? *(please use the 128-example subset of ImageNet used earlier)*. If you observe an improvement discuss why this is the case.
  * Comment on the similarity and design considerations of your modified version of AlexNet to the MobileNet architecture discussed in lectures.
  


## C.4: Investigating Gradient Stabilization

As Ferenc mentioned during his lectures gradient instability is a fundemental barrier to successful training. In this mini-project we will dive into this issue. 

Before beginning, construct a version of MobileNetV2 from which you have removed the architectural components that manage the gradient. That is remove from it batch normalization layers and residual connections. We will use this "stripped" version of MobileNetV2 multiple times below (we call it below "stripped MobileNetV2"). You will recall this architecture from the prior assignment. 

*As a hint: you will also find it useful to implement a helper function that computes the average gradient at each layer when provided a batch of training examples.*

* Early Training Gradient Stabilization *(~20 marks).*
  * Use the stripped MobileNetV2 described above and begin by initalizing it (rather than using any pretrained weights). Create three versions related to three initialization schemes: (1) all parameters set to a constant; (2) all set as random uniform; (3) all set using Xavier initialization (i.e., use the default PyTorch initialization). (For some background on this topic consult this [paper](https://arxiv.org/abs/1704.08863).)
  * Visualize the gradients generated when training these three networks using one batch of data taken from ImageNet, holding everything else constant. *(note, use the visualization helper function described earlier)*. Briefly comment on the differences you see. Expand your set of experiments by also testing with at least three different constants for the first scheme described (e.g., try using: $0$, $1$, $1000$).
* Mid-Training Gradient Stabilization *(~40 marks).*
  * We will now investigate gradients that are in-use during *mid-training*. These gradients will be available if you take the original MobileNetV2 architecture amd train it for three epochs using ImageNet using all the settings (e.g., initalizer etc.) detailed in the original [paper](https://arxiv.org/abs/1801.04381). Copy the weights of this network to the "stripped" MobileNetV2.  
  * Investigate the role played by the residual connections. Insert the residual connections back to the model, and backpropagate a single batch (ImageNet). Collect and visualize the gradients using your helper function. Comment on what are the effects of inserting the residual connections back to the early, middle, and the final layers of the network. 
  * Next, investigate the role played by the batch normalization. Insert the batch normalization layers back to the model, and backpropagate a single batch through the model. Collect and visualize the gradients using your helper function. Comment on what effects does inserting the batch normalization back to the beginning, middle, and the end of the network have. *(note, use the batch normalization in its training mode)*.
  * Compare the effects of the residual connections and the batch normalizations.
* Gradients - Going Beyond Training *(~20 marks).* 
  * For this final exploration, revert back to a pretrained MobileNetV2 without any changes to the architecture. In the following steps we will freeze the model's parameters, and propagate gradients into the input.
  * Define a simple loss function that is minimized when the output activation corresponding to one ImageNet class is maximized and all other classes have 0 value. (Let's call this class $C$, you can select any ImageNet class you wish). 
  * Given this loss, now identify an input image that would minimize the loss. Perform a test where you initialize an input in four different ways: (1) all zeros; (2) random; (3) image of the class $C$; (4) image not of the class $C$. Compute the gradients of the loss function with respect to the pixels in the input image. *(note, the network parameters are held constant!)* Update the image using these gradients. Repeat gradient computation and update until image stops changing.
  * Comment on training the image. What kind of image did you arrive at -- is it representative of the class you were training towards? How did the image change as you were "training" it? And, what difference did the initialization strategies make - did you arrive at different final images and if so then why? If not, then why not?